<a href="https://colab.research.google.com/github/abdul0125/Fake-News-Prediction/blob/main/Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the dependencies

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
#Loading dataset
news_dataset = pd.read_csv('/content/FakeNews.csv',engine='python', error_bad_lines=False)



/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 11630: unexpected end of data


In [ ]:
news_dataset.shape


(11628, 5)

In [ ]:
news_dataset.tail()

,id,title,author,text,label
11623,11623,The best halloween costume ever is this guy wh...,Sausage Machine,Next Prev Swipe left/right The best halloween ...,1
11624,11624,What If….,Karl Denninger,"all 30,000 of the “missing” Hillary emails are...",1
11625,11625,"Chaiwali, an Indian Restaurant That Feels Like...",Pete Wells,"Greeting us when we walked into Chaiwali, the ...",0
11626,11626,‘Nothing Short of a Miracle’: No Serious Injur...,Eli Rosenberg,In the ambitious business of New York bridge b...,0
11627,11627,What You Should Watch: ‘Please Like Me’ and Fi...,Margaret Lyons,"Welcome to Watching, The New York Times’s g...",0


In [ ]:
#Counting the number o fmissing values in the dataset
news_dataset.isnull().sum()

id           0
title      316
author    1088
text        25
label        0
dtype: int64

In [ ]:
#Filling null values with empty strings
news_dataset = news_dataset.fillna('')

In [ ]:
news_dataset.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [ ]:
#Merging author and title colums together
news_dataset["content"] = news_dataset["author"]+" "+news_dataset["title"]

In [ ]:
news_dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [ ]:
#Separating labels and content
X = news_dataset.drop(columns="label",axis=1)
Y=news_dataset["label"]

In [ ]:
print(X,Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
11623  11623  The best halloween costume ever is this guy wh...   
11624  11624                                          What If….   
11625  11625  Chaiwali, an Indian Restaurant That Feels Like...   
11626  11626  ‘Nothing Short of a Miracle’: No Serious Injur...   
11627  11627  What You Should Watch: ‘Please Like Me’ and Fi...   

                   author                                               text  \
0           Darrell Lucus  House Dem Aide: We Didn’t Even See Comey’s Let...   
1         Daniel J. Flynn  Ever get

#Stemming - stemming is a process of converting a word to its root word

In [ ]:
port_stem =PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content



In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
news_dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


In [ ]:
X=news_dataset['content'].values
Y=news_dataset['label'].values

In [ ]:
print(X,Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'pete well chaiwali indian restaur feel like home new york time'
 'eli rosenberg noth short miracl seriou injuri tappan zee crane collaps new york time'
 'margaret lyon watch pleas like film leav netflix new york time'] [1 0 1 ... 0 0 0]


#Converting textual data to numerical data

In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X) 

In [ ]:
print(X)

  (0, 12145)	0.28579863399464217
  (0, 10413)	0.2625416814135292
  (0, 6907)	0.3573488791146282
  (0, 6693)	0.29011709663150426
  (0, 5978)	0.24626195783302973
  (0, 5452)	0.22521823987158263
  (0, 3884)	0.23805393759579274
  (0, 2975)	0.2711338079825989
  (0, 2823)	0.3573488791146282
  (0, 2319)	0.24836718585022602
  (0, 1950)	0.3631105737863691
  (0, 225)	0.2733855509628368
  (1, 12963)	0.2950142861761186
  (1, 5316)	0.18786143273795808
  (1, 4308)	0.7203975277737739
  (1, 2799)	0.26509360027800555
  (1, 2201)	0.18884393269982766
  (1, 1749)	0.38259211638857626
  (1, 1501)	0.15164486918670209
  (1, 1194)	0.2887776191463554
  (2, 12091)	0.4256848875864237
  (2, 7440)	0.4879246436151162
  (2, 4659)	0.3459730696851783
  (2, 4221)	0.38513297573901556
  (2, 2436)	0.4585220163282515
  :	:
  (11626, 13150)	0.33938993653199
  (11626, 13115)	0.080783508115348
  (11626, 11841)	0.07937110143438547
  (11626, 11622)	0.33938993653199
  (11626, 10642)	0.2717575264094761
  (11626, 10487)	0.308344336

In [ ]:
#splitting data into test and training data
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression()

In [ ]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [ ]:
print(training_data_accuracy)

0.9822618791657708


In [ ]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [ ]:
print(test_data_accuracy)

0.9707652622527945


#Building predictive Model

In [ ]:
X_random = X_test[0]

prediction = model.predict(X_random)

if prediction[0]==0:
  print("predicted result is ","real news")
else:
  print("predicted result is","Fake news")



predicted result is Fake news


In [ ]:
print(Y_test[0])

1
